# Applied Project in Big Data on Industrial Dataset

## MODELS SELECTION TECHNIQUES
## Part II. Straight way to organize a project and track experiments

### 1. Libraries

In [ ]:
import os
import re
import json
import time
import random
import datetime
import numpy as np
import pandas as pd
from joblib import dump, load
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
from sklearn.feature_extraction.text import (
    TfidfVectorizer, 
    CountVectorizer
)
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score, 
    confusion_matrix, 
    precision_score, 
    recall_score, 
    f1_score,
    roc_curve, 
    auc
)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    cross_val_score, 
    train_test_split,
    StratifiedKFold
)
pd.set_option('display.max_columns', None)
N_CORES = min(
    multiprocessing.cpu_count(), 
    int(float(os.environ['CPU_LIMIT']))
)
print('cores:', N_CORES)

### 2. Create config and place to store artifacts

In [ ]:
start_time = time.time()

VER = 'v1'
CONFIG = {
    'version': VER,
    'start_time': str(datetime.datetime.fromtimestamp(start_time)),
    'sample_size': 1000,
    'ngram_range': (1, 1), 
    'max_df': .95, 
    'min_df': 10,
    'clf': 'RandomForestClassifier',
    'folds': 8,
    'seed': 2021,
    'n_iters': 50,
    'comments': ''
}
MDLS_PATH = f'./models_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/config.json', 'w') as file:
    json.dump(CONFIG, file)
    
def seed_all(seed):
    """
    Sometimes it is useful to nail all randomness
    and fix all random seeds for reproducibility.
    
    This function fixes all random seeds for current pipline, 
    but it could be extended e.g. for Tensorflow library 
    you may want to add `tf.random.set_seed(seed)` in the code.
    
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_all(CONFIG['seed'])

### 2. Create a small dataset for demo

In [ ]:
df = pd.read_csv('data_white_en.csv')
df = df.sample(CONFIG['sample_size']).reset_index()
del df['index']
df['label'] = 0
print(df.shape)
display(df.head())

In [ ]:
df_tmp = pd.read_csv('data_black_en.csv')
df_tmp = df_tmp.sample(CONFIG['sample_size']).reset_index()
del df_tmp['index']
df_tmp['label'] = 1
print(df_tmp.shape)
display(df_tmp.head())

In [ ]:
df = df.append(df_tmp)
df.reset_index(inplace=True)
del df['index']
print(df.shape)
display(df.head())

In [ ]:
# not necessary
save_data_path = f'{MDLS_PATH}/data_{CONFIG["version"]}.csv'
df.to_csv(save_data_path)

### 3. Modelling with save

In [ ]:
def text_features(data, vectorizer):
    print('total texts:', len(data))
    features = vectorizer.fit_transform(data)
    print(
        'features shape:', features.shape, 
        'max:', np.max(features), 
        'min:', np.min(features)
    )
    return features, vectorizer

In [ ]:
def cross_val_model(X, y, 
                    folds, clf,
                    vectorizer, ngram_range=(1, 1), 
                    max_df=.2, min_df=8, seed=2022):
    scores = {}
    roc_auc_scores = []
    f1_scores = []
    skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)
    for fold, (train_idxs, test_idxs) in enumerate(skf.split(X, y)):
        
        # train model
        
        X_train, X_test = X.iloc[train_idxs], X.iloc[test_idxs]
        y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
        X_train, vectorizer = text_features(
            X_train, 
            vectorizer=vectorizer
        )
        X_test = vectorizer.transform(X_test)
        clf.fit(X_train, y_train)
        
        # saving models
        # more about https://scikit-learn.org/stable/model_persistence.html
        # NOTE - not only model, but vectorizer too!
        
        file_name = f'{MDLS_PATH}/model_fold_{fold}.joblib'
        dump(clf, file_name)
        print('saved to', file_name)
        file_name = f'{MDLS_PATH}/vectorizer_fold_{fold}.joblib'
        dump(vectorizer, file_name)
        print('saved to', file_name)
        
        # metrics
        
        y_score = clf.predict_proba(X_test)
        roc_auc_score_ = roc_auc_score(y_test, y_score[:, 1])
        roc_auc_scores.append(roc_auc_score)
        y_pred = clf.predict(X_test)
        f1_score_ = f1_score(y_test, y_pred)
        f1_scores.append(f1_score_)
        msg = f'fold {fold} - val ROC-AUC score: {roc_auc_score_:.2f}, val f1-score: {f1_score_:.2f}'
        print(msg)
        
        scores[f'fold {fold}'] = {
            'roc_auc_scores': roc_auc_scores,
            'f1_scores': f1_scores
        }
        
        # logging (if needed)
        # read about logging for production cases here https://docs.python.org/3/library/logging.html
        
        log_file_name = f'{MDLS_PATH}/model_{CONFIG["version"]}.log'
        log_string = f'{str(datetime.datetime.fromtimestamp(time.time()))} - {msg}\n'
        with open(log_file_name, 'a') as file:
            file.write(log_string)
        
    return scores

In [ ]:
vectorizer=TfidfVectorizer(
    ngram_range=CONFIG['ngram_range'], 
    max_df=CONFIG['max_df'], 
    min_df=CONFIG['min_df']
)
if CONFIG['clf'] == 'RandomForestClassifier':
    clf = RandomForestClassifier(n_estimators=CONFIG['n_iters']) 
else:
    clf = LogisticRegression()
scores = cross_val_model(
    X=df['proc'], 
    y=df['label'], 
    folds=CONFIG['folds'], 
    clf=clf,
    vectorizer=vectorizer,
    seed=CONFIG['seed']
)

### 4. Not only save models but get them back

In [ ]:
VER = 'v0'
with open(f'{MDLS_PATH}/config.json', 'r') as file:
    config = json.load(file)
print(config)

In [ ]:
models = []
vectorizers = []
for fold in range(config['folds']):
    file_name = f'{MDLS_PATH}/model_fold_{fold}.joblib'
    model = load(file_name)
    models.append(model)
    
    file_name = f'{MDLS_PATH}/vectorizer_fold_{fold}.joblib'
    vectorizer = load(file_name)
    vectorizers.append(vectorizer)

In [ ]:
models

In [ ]:
vectorizers

In [ ]:
df_tmp = pd.read_csv('data_black_en.csv')
df_tmp = df_tmp.sample(100).reset_index()
del df_tmp['index']
df_tmp['label'] = 1
print(df_tmp.shape)
display(df_tmp.head())

In [ ]:
test_vectorizer = vectorizers[0]

In [ ]:
X_test = test_vectorizer.transform(df_tmp['proc'])

In [ ]:
test_model = models[0]

In [ ]:
y_score = test_model.predict_proba(X_test)

In [ ]:
df_tmp['predictions'] = y_score[:, 1]
print(df_tmp.shape)
display(df_tmp.head())

### 5. Way of Jedi datascientist - make a predictions

In [ ]:
def jedi_predictions(data, models, vectorizers):
    preds = []
    for m, v in zip(models, vectorizers):
        data_ = v.transform(data)
        y_score = m.predict_proba(data_)
        preds.append(y_score)
    return np.mean(preds, axis=0)

In [ ]:
y_score = jedi_predictions(df_tmp['proc'], models, vectorizers)
y_score[:5, :]

In [ ]:
df_tmp['predictions'] = y_score[:, 1]
print(df_tmp.shape)
display(df_tmp.head())